# Quantum Imaginary Time Evolution of Phenanthrene-Quinone Fragment (Carbonyl Formation)

# Introduction

**Phenanthrenequinone** (9,10-phenanthrenedione) is an oxidized fragment of phenanthrene featuring two carbonyl groups at the 9 and 10 positions. This molecule serves as a model for studying **carbonyl formation in aromatic systems**, such as oxidative aging of hydrocarbons.

In this notebook, we use the **Tangelo** quantum chemistry framework to simulate the ground state of the phenanthrenequinone fragment. We will employ **Quantum Imaginary Time Evolution (QITE)** – an algorithm that iteratively evolves a trial state in imaginary time to project out the ground state – and map the fermionic molecular Hamiltonian to qubits using the **Jordan–Wigner (JW)** transformation.

To make the simulation tractable, we use a minimal **STO-3G** basis set and apply **active space reduction techniques** (frozen core and selected orbitals), as recommended for phenanthrene. By freezing core orbitals (e.g., carbon and oxygen 1s) and focusing on the frontier molecular orbitals involved in the carbonyl functionality, we drastically reduce the number of qubits required while preserving the key chemistry of the reactive fragment.

The **JW mapping** encodes the fermionic Hamiltonian into a qubit operator representation, which QITE uses to find the ground-state energy. We will compare the **QITE-computed ground-state energy** to the baseline **Hartree–Fock** energy, highlighting:

- the **correlation energy** captured by QITE, and  
- its chemical significance (e.g., insight into **reactivity** and **multi-reference character**).


In [17]:
try:
    import tangelo
except ImportError:
    !pip install git+https://github.com/sandbox-quantum/Tangelo.git@develop --quiet
    !pip install pyscf --quiet

!pip install pyscf
from pyscf import __config__
__config__.B3LYP_WITH_VWN5 = True

# Pretty printer for more readable outputs
import pprint
pp = pprint.PrettyPrinter(width=160, compact=False, indent=1)
from pprint import pprint

# Installation of tangelo if not already installed.
try:
    import tangelo
except ModuleNotFoundError:
    !pip install git+https://github.com/sandbox-quantum/Tangelo.git@develop --quiet

## Molecular Geometry and Basis Set

We define the **phenanthrene-9,10-quinone** fragment using its full Cartesian geometry (24 atoms: C₁₄H₈O₂). The XYZ coordinates below (in angstroms) correspond to an optimized 3D structure of *9,10-phenanthrenedione*. 

We use the minimal **STO-3G** basis set for initial exploration, which provides one basis function per atomic orbital. This choice keeps the problem size manageable, at the cost of some accuracy.

In [8]:
# Define the molecular geometry (phenanthrene-9,10-dione fragment) in XYZ format
phenanthrenequinone_xyz = """24
Phenanthrene-9,10-dione (phenanthrenequinone) geometry in Cartesian coordinates (angstroms)
O   -1.9481    2.8035   -0.5304
C   -1.1071    1.8848   -0.3547
C    0.3136    2.1983   -0.3990
O    0.7324    3.3780   -0.6108
C    1.2762    1.1045   -0.1902
C    2.6344    1.3575   -0.2239
C    3.5449    0.3474   -0.0309
C    3.0762   -0.9115    0.1949
C    1.7173   -1.1595    0.2277
C    0.7755   -0.1731    0.0388
C   -0.6481   -0.4582    0.0778
C   -1.1144   -1.7414    0.3081
C   -2.4884   -1.9829    0.3396
C   -3.4152   -0.9787    0.1475
C   -2.9552    0.2852   -0.0793
C   -1.5891    0.5314   -0.1117
H    2.9676    2.3856   -0.4088
H    4.5992    0.6008   -0.0675
H    3.7651   -1.7243    0.3498
H    1.3436   -2.1705    0.4091
H   -0.4895   -2.5802    0.4672
H   -2.8910   -2.9734    0.5170
H   -4.4825   -1.1461    0.1683
H   -3.6173    1.1230   -0.2385
"""

We will create a Tangelo `SecondQuantizedMolecule` with this geometry, specifying a neutral charge (`q=0`) and singlet spin (`spin=0`, meaning all 108 electrons are paired). Tangelo uses **PySCF** under the hood to perform a **Hartree–Fock (HF)** calculation and obtain the molecular orbitals and integrals.

## Active Space Selection and Frozen Core Approximation

**Phenanthrenequinone** is a fairly large system in a full **STO-3G** basis (88 MOs, which would translate to 176 qubits if taken fully). 

To reduce complexity, we apply the **frozen core approximation**, freezing the low-energy core orbitals (the 1s orbitals on each carbon and oxygen). These core orbitals (16 in total for C₁₄O₂) are doubly occupied and chemically inert, so we exclude them from the active quantum simulation.

In addition, we restrict the **active space** to a selection of frontier orbitals around the **highest occupied** and **lowest unoccupied molecular orbitals (HOMO and LUMO)**. This focuses the simulation on the π-system and carbonyl orbitals relevant to reactivity.

We will keep the **HOMO, LUMO**, and a few neighboring orbitals (e.g., HOMO–3 through LUMO+3) in the active space, and freeze all other valence orbitals. This yields a manageable active space of around **8 MOs (16 spin-orbitals)** for the QITE simulation, dramatically reducing the qubit count while retaining key correlation effects.

Using **Tangelo’s utilities**, we can determine which orbitals to freeze. First, we initialize the full molecule and compute the **mean-field solution**. Then we use helper functions to get a freeze list that excludes all orbitals except the chosen HOMO/LUMO window.

In [20]:
from tangelo import SecondQuantizedMolecule
from tangelo.toolboxes.molecular_computation.frozen_orbitals import get_frozen_core, get_orbitals_excluding_homo_lumo

# Initialize the full molecule (perform HF calculation)
full_mol = SecondQuantizedMolecule(phenanthrenequinone_xyz, q=0, spin=0, basis="STO-3G")
print(f"Total MOs (full): {full_mol.n_mos}, Electrons: {full_mol.n_electrons}")

# Determine frozen core orbitals and active space around HOMO/LUMO
n_core = get_frozen_core(full_mol)  # number of core orbitals to freeze (int)
freeze_list = get_orbitals_excluding_homo_lumo(full_mol, homo_minus_n=3, lumo_plus_n=3)
print(f"Number of core orbitals = {n_core}")
print(f"Frozen orbital indices (excl. HOMO-3..LUMO+3 active space): {freeze_list}")

ModuleNotFoundError: No module named 'tangelo'

In the output, `full_mol.n_mos` is the total number of molecular orbitals (88 for full STO-3G here), and `full_mol.n_electrons` confirms the electron count (108 for C₁₄H₈O₂).

The `get_frozen_core` function returns the count of lowest-energy occupied MOs to freeze (which should equal 16 in this case, corresponding to the carbon and oxygen 1s orbitals).

The `get_orbitals_excluding_homo_lumo` function returns a list of orbital indices that lie outside the specified HOMO±3 window – i.e., these will be frozen.

Next, we construct a new `SecondQuantizedMolecule` with the specified frozen orbitals:

In [21]:
# Build the active-space molecule with core and certain valence orbitals frozen
active_mol = SecondQuantizedMolecule(phenanthrenequinone_xyz, q=0, spin=0, basis="STO-3G", frozen_orbitals=freeze_list)
print(f"Active space MOs: {active_mol.n_mos}, Active electrons: {active_mol.n_electrons}")

NameError: name 'SecondQuantizedMolecule' is not defined

Now, `active_mol` represents the **reduced Hamiltonian** in the active space (core removed, only HOMO–3 to LUMO+3 unfrozen). 

The printed output shows the number of **active MOs** and **active electrons**. We expect considerably fewer MOs than 88 — specifically, if 16 core orbitals and a number of additional valence orbitals are frozen, the active count might be on the order of **8 MOs** (for HOMO–3 to LUMO+3, which is 4 occupied + 4 virtual = 8 orbitals).

The number of **active electrons** would be the count of electrons in those active orbitals. For example, 8 orbitals with maybe 16 electrons if 4 of them were occupied in the Hartree–Fock solution.

## Hamiltonian Construction and Qubit Mapping (Jordan–Wigner)

With the active-space `SecondQuantizedMolecule`, we can obtain the **molecular electronic Hamiltonian** in second-quantized form (fermionic operators) and then map it to a **qubit Hamiltonian** via the **Jordan–Wigner (JW) transformation**. 

Tangelo internally interfaces with **OpenFermion** to represent operators. We can access the fermionic Hamiltonian as an OpenFermion `FermionOperator` and apply the JW mapping to get a `QubitOperator` (Pauli-sum representation).

The output will indicate:
- The number of **Pauli terms** in the qubit Hamiltonian
- The number of **qubits** (`active_mol.n_qubits`), which should equal twice the number of active MOs (since each molecular orbital contributes two spin-orbitals/qubits in JW mapping)

For example, an **8-orbital active space** corresponds to **16 qubits**.

The **Jordan–Wigner mapping** encodes each fermionic creation/annihilation operator into a string of **Pauli-$Z$ and Pauli-$X/Y$** operators on qubits, preserving the correct fermionic commutation relationships. This comes at the cost of introducing long strings of $Z$ operators to account for fermion parity.

*Optional:* We could examine a snippet of `qubit_hamiltonian` to see its structure (e.g., a sum of Pauli strings like `Z0 Z1 ... - 0.1234`), but it’s generally quite lengthy. For brevity, we proceed to the simulation.

In [22]:
from tangelo.toolboxes.operators import QubitOperator

# Retrieve the second-quantized (fermionic) Hamiltonian
fermion_hamiltonian = active_mol.fermionic_hamiltonian

# Apply the Jordan-Wigner transform to get qubit Hamiltonian
# (Tangelo uses OpenFermion's mapping under the hood)
qubit_hamiltonian = fermion_hamiltonian.jordan_wigner()  
print(f"Qubit Hamiltonian has {len(qubit_hamiltonian.terms)} terms acting on {active_mol.n_qubits} qubits.")

ModuleNotFoundError: No module named 'tangelo'